In [1]:
import requests
import pandas as pd
import os
import re


In [2]:
df_donosti = pd.read_csv('data/Datos_Aemet/donosti_csv/datos_donosti.csv', sep =",")
df_donosti

,fecha,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
0,1970-1,9.5,12.1,6.8,18.0(19),-1.4(07),5.0,13.0,196.7,2.0,0.0,2.4
1,1970-2,7.7,9.9,5.5,14.0(04),-2.0(28),3.0,11.0,165.8,0.0,0.0,2.3
2,1970-3,7.2,10.0,4.3,20.6(21),-1.2(02),2.1,9.5,101.0,0.0,0.0,2.6
3,1970-4,9.5,12.5,6.5,26.5(16),1.2(03),6.9,10.9,174.3,1.0,0.0,3.9
4,1970-5,14.0,17.4,10.4,30.6(24),5.0(08),9.4,15.0,89.4,1.0,1.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...
684,2022-9,18.6,22.1,15.1,33.9(11),10.3(25),14.0,22.5,147.6,2.0,4.0,6.4
685,2022-10,19.6,22.9,16.3,28.0(19),11.8(01),17.7,24.3,40.6,0.0,0.0,4.1
686,2022-11,13.4,16.5,10.2,22.0(13),6.4(19),9.5,17.5,317.3,4.0,0.0,3.6
687,2022-12,11.9,14.7,9.0,20.0(31),2.0(10),7.4,15.7,82.4,0.0,0.0,2.9


In [3]:
#Separamos la columna "fecha" con año-mes en dos
df_donosti[['año', 'mes']] = df_donosti['fecha'].str.split('-', expand=True)
# Eliminamos la columna "fecha"
df_donosti2 = df_donosti.drop(columns=["fecha"], axis=1)
# Ponemos las nuevas columnas año, mes las primeras
column_order = ['año', 'mes'] + df_donosti2.columns.tolist()[:-2]
df_donosti3 = df_donosti2.reindex(columns=column_order)
df_donosti3
df_donosti3['ta_max'] = df_donosti3['ta_max'].str.replace(r'\(.*\)', '', regex=True)
df_donosti3['ta_min'] = df_donosti3['ta_min'].str.replace(r'\(.*\)', '', regex=True)
df_donosti3
# Así se pasan las filas de los datos anuales a otro dataframe
df_donosti_anual = pd.DataFrame(df_donosti3.iloc[12::13])
df_donosti_anual
# Y eliminamos esas filas del dataframe original
df_donosti_final = df_donosti3.drop(range(12,663,13), axis=0)
df_donosti_final
#Generamos csv y xlsx para guardar los archivos y que no se pierda el trabajo
df_donosti_anual.to_csv('data/csv_creados/donosti_anual.csv', index=False)
df_donosti_final.to_csv('data/csv_creados/donosti_final.csv', index=False)

In [4]:
df_donosti_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 638 entries, 0 to 688
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     638 non-null    object 
 1   mes     638 non-null    object 
 2   tm_mes  638 non-null    float64
 3   tm_max  638 non-null    float64
 4   tm_min  638 non-null    float64
 5   ta_max  638 non-null    object 
 6   ta_min  638 non-null    object 
 7   ti_max  638 non-null    float64
 8   ts_min  638 non-null    float64
 9   p_mes   638 non-null    float64
 10  np_300  638 non-null    float64
 11  nt_30   638 non-null    float64
 12  inso    632 non-null    float64
dtypes: float64(9), object(4)
memory usage: 69.8+ KB


In [4]:
df_donosti_final['inso'] = df_donosti_final['inso'].fillna(0)
df_donosti_final_sinull = df_donosti_final
df_donosti_final_sinull.reset_index(drop=True, inplace=True)
df_donosti_final_sinull.to_csv('data/csv_creados/donosti_final_sinull.csv', index=False)
df_donosti_final_sinull

,año,mes,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
0,1970,1,9.5,12.1,6.8,18.0,-1.4,5.0,13.0,196.7,2.0,0.0,2.4
1,1970,2,7.7,9.9,5.5,14.0,-2.0,3.0,11.0,165.8,0.0,0.0,2.3
2,1970,3,7.2,10.0,4.3,20.6,-1.2,2.1,9.5,101.0,0.0,0.0,2.6
3,1970,4,9.5,12.5,6.5,26.5,1.2,6.9,10.9,174.3,1.0,0.0,3.9
4,1970,5,14.0,17.4,10.4,30.6,5.0,9.4,15.0,89.4,1.0,1.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,2022,9,18.6,22.1,15.1,33.9,10.3,14.0,22.5,147.6,2.0,4.0,6.4
634,2022,10,19.6,22.9,16.3,28.0,11.8,17.7,24.3,40.6,0.0,0.0,4.1
635,2022,11,13.4,16.5,10.2,22.0,6.4,9.5,17.5,317.3,4.0,0.0,3.6
636,2022,12,11.9,14.7,9.0,20.0,2.0,7.4,15.7,82.4,0.0,0.0,2.9


In [5]:
df_donosti_final_sinull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     638 non-null    object 
 1   mes     638 non-null    object 
 2   tm_mes  638 non-null    float64
 3   tm_max  638 non-null    float64
 4   tm_min  638 non-null    float64
 5   ta_max  638 non-null    object 
 6   ta_min  638 non-null    object 
 7   ti_max  638 non-null    float64
 8   ts_min  638 non-null    float64
 9   p_mes   638 non-null    float64
 10  np_300  638 non-null    float64
 11  nt_30   638 non-null    float64
 12  inso    638 non-null    float64
dtypes: float64(9), object(4)
memory usage: 64.9+ KB


In [7]:
df_donosti_anual['inso'] = df_donosti_anual['inso'].fillna(0)
df_donosti_anual_sinull = df_donosti_anual
df_donosti_anual_sinull2=df_donosti_anual_sinull.drop(["mes"], axis=1)
df_donosti_anual_sinull2.reset_index(drop=True, inplace=True)
df_donosti_anual_sinull2.to_csv('data/csv_creados/donosti_anual_sinull2.csv', index=False)
df_donosti_anual_sinull2.head()

,año,tm_mes,tm_max,tm_min,ta_max,ta_min,ti_max,ts_min,p_mes,np_300,nt_30,inso
0,1970,13.1,16.1,10.1,34.6,-2.0,1.8,21.6,1441.5,9.0,5.0,4.4
1,1971,12.9,16.0,9.8,32.2,-5.5,1.0,23.0,1594.0,4.0,4.0,4.4
2,1972,12.3,15.4,9.2,32.2,-1.3,5.0,19.0,1366.8,6.0,2.0,4.4
3,1973,12.8,16.0,9.6,33.0,-6.7,2.0,22.0,1532.1,8.0,5.0,4.5
4,1974,12.5,15.7,9.3,36.4,0.0,5.3,21.8,1840.2,8.0,4.0,4.0


In [8]:
df_donosti_anual_sinull2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     53 non-null     object 
 1   tm_mes  53 non-null     float64
 2   tm_max  53 non-null     float64
 3   tm_min  53 non-null     float64
 4   ta_max  53 non-null     object 
 5   ta_min  53 non-null     object 
 6   ti_max  53 non-null     float64
 7   ts_min  53 non-null     float64
 8   p_mes   53 non-null     float64
 9   np_300  53 non-null     float64
 10  nt_30   53 non-null     float64
 11  inso    53 non-null     float64
dtypes: float64(9), object(3)
memory usage: 5.1+ KB
